<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter3 Attentionメカニズムのコーディング

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [ ]:
# 2つ目のトークンをクエリとして使う
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # ドット積
print(attn_scores_2)

### ドット積：

ドット積とは、基本的には、2つのベクトルの要素ごとに掛け合わせ、その積の総和を求める簡潔な方法

2つのベクトルを結合してスカラー値を得るための数学的な手段であるだけでなく、ベクトル同士の類似度の尺度でもある。なぜなら、2つのベクトルがどれくらい密に並んでいるか（どのくらい同じ方向を向いているか）を定量化するからである。ベクトル間のドット積が大きいほど、ベクトル間の類似度は高くなる。

ドット積はシーケンスの各要素が他の要素にどれくらい注目しているか（注意を払っている度合い）を決定する。ドット積が大きいほど、2つの要素は類似していると見なされ、それらの間のAttentionスコアも高くなる。

In [ ]:
# ドット積
res = 0
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

# 要素ごとの積の和が、ドット積（torch.dot()）と同じになる
print(res)
print(torch.dot(inputs[0], query))

In [ ]:
# Attentionスコアの正規化 -> 総和が1になるようにする
# 正規化することで、LLMにおいて重みの解釈を容易にし、学習の安定性を維持するのに役立つ慣例となる
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights: ", attn_weights_2_tmp)
print("Sum: ", attn_weights_2_tmp.sum())

In [ ]:
# 正規化にはソフトマックス関数を使用するのが一般的
# 極端な値を上手く扱うのに適しており、訓練時の勾配特性をより安定させる
# ソフトマックス関数では、Attentionの重みが常に正になる

def softmax_navive(x):
    return torch.exp(x) / torch.exp(x).sum()

attn_weights_2_navive = softmax_navive(attn_scores_2)
print("Attention weights: ", attn_weights_2_navive)
print("Sum: ", attn_weights_2_navive.sum())

In [ ]:
# torchのsoftmax関数（これ使う）
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights: ", attn_weights_2)
print("Sum: ", attn_weights_2.sum())

In [ ]:
# 得られたベクトルをすべて加算する（加重和）
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i # 1x5 * 5x3 = 1x3 の行列が出力
print(context_vec_2)

In [ ]:
# すべての入力のペアについてドット積を計算
# Step1: Attentionスコアを計算
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

In [ ]:
# 行列積を使えば簡単に求まるよ
attn_scores = inputs @ inputs.T
print(attn_scores)

In [ ]:
# Step2: SoftMax関数で正規化 -> Attentionの重みを求める
# 入力パラメータdim: 入力テンソルのどの次元に沿って計算を行うか
# dim=-1に設定すると、入力テンソルの最後の次元に沿って正規化するようになる
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

In [ ]:
# 2次元テンソルは列ごとに正規化が行われ、各行（列次元）値の合計が1になる
row_2_sum = sum(attn_weights[1])
print("Row 2 sum: ", row_2_sum)
print("All row sums: ", attn_weights.sum(dim=-1))

In [ ]:
# Step3: Step2のAttentionの重みと入力テンソルの行列積を使って、
# すべてのコンテキストベクトルを計算
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

In [ ]:
# 前に計算した2行目の値と比較
print("Previous 2nd context vector: ", context_vec_2)